In [1]:
import sys
sys.path.insert(0,'/Users/jacobgollub/Desktop/college/research/pbp_explorations/scripts/sackmann')
import tennisGameProbability,tennisMatchProbability,tennisSetProbability,tennisTiebreakProbability
from tennisMatchProbability import matchProb
from helper_functions import *
from data_functions import generate_elo_induced_s
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss,accuracy_score

FILE_NAME = 'elo_atp_matches_all_10_2.csv'
# can test this on our subset of 10,000 matches as well as all matches in the database:
df = pd.read_csv('../my_data/'+FILE_NAME)
del df['Unnamed: 0']

# currently looking at 2014 tour-level matches, excluding Davis Cup
#df = df[df['match_year']==2014].reset_index(drop=True)
#df = df[df['match_year'].isin([2011,2012,2013,2014,2015])]
df = df[df['tny_name']!='Davis Cup'].reset_index(drop=True)
df.head(2)

,tny_id,tny_name,surface,tny_date,match_year,match_month,p0_name,p1_name,p0_elo,p1_elo,...,match_prob_sf_kls_JS,match_prob_adj_kls,match_prob_adj_kls_JS,elo_prob,elo_prob_538,sf_elo_prob,sf_elo_prob_538,s_total,p0_s_kls_elo,p1_s_kls_elo
0,2000-339,Adelaide,Hard,2000-01-03,2000,1,Arnaud Clement,Thomas Enqvist,1675.243826,1921.802173,...,0.354126,0.223227,0.224541,0.194771,0.204867,0.211304,0.233946,1.238109,0.585200,0.652909
1,2000-339,Adelaide,Hard,2000-01-03,2000,1,Roger Federer,Jens Knippschild,1664.533683,1697.781488,...,0.476991,0.431598,0.432470,0.452298,0.518498,0.546058,0.594731,1.287473,0.638864,0.648608


In [13]:
len(sub[sub['elo_diff']>0]),len(sub[sub['elo_diff']<0])

(9705, 8540)

In [7]:
sub = df[df['match_year']>=2010]
s = (len(sub[(sub['elo_diff']>0) & (sub['winner'])])+len(sub[(sub['elo_diff']<0) & (sub['winner']==0)]))
s /= float(len(sub))
s

0.6899934239368698

In [2]:
from scipy.stats import pearsonr
x = pearsonr(df['match_prob_adj_kls_JS'],df['elo_prob'])[0]
x**2

0.76071848681709175

In [3]:
# df = df[df['match_year']==2014]
# print 'elo baseline: ',  sum((df['elo_diff']>0) == df['winner'])/float(len(df))
# print log_loss(df['winner'],[(1+10**(diff/-400.))**-1 for diff in df['elo_diff']])
# print log_loss(df['winner'],[(1+10**(diff/-400.))**-1 for diff in df['sf_elo_diff']])
# print 'surface elo baseline: ', sum((df['sf_elo_diff']>0) == df['winner'])/float(len(df))
# print 'elo 538 baseline: ',  sum((df['elo_diff_538']>0) == df['winner'])/float(len(df))
# print log_loss(df['winner'],[(1+10**(diff/-400.))**-1 for diff in df['elo_diff_538']])
# print log_loss(df['winner'],[(1+10**(diff/-400.))**-1 for diff in df['sf_elo_diff_538']])
# print 'surface elo 538 baseline: ', sum((df['sf_elo_diff_538']>0) == df['winner'])/float(len(df))

In [4]:
cols = [['elo_diff'],['sf_elo_diff'],['elo_diff_538'],['sf_elo_diff_538'],
        ['elo_diff','sf_elo_diff'],['elo_diff_538','sf_elo_diff_538']]
#         ['elo_diff','sf_elo_diff','match_z_kls'],\
#         ['elo_diff_538','sf_elo_diff_538','match_z_kls']]
probs = [u'match_prob_kls',u'match_prob_kls_JS', u'match_prob_sf_kls', u'match_prob_sf_kls_JS',
        'match_prob_adj_kls','match_prob_adj_kls_JS']
n_splits = 5
train_df,test_df = df[df['match_year']!=2014],df[df['match_year'].isin([2014,2015])]
#validate_results(df,probs=probs,lm_columns=cols,n_splits=n_splits)
y_logit_probs = test_results(train_df,test_df,probs=probs,lm_columns=cols)

match_prob_kls
accuracy:  0.649020004041
loss:  0.641058120068
match_prob_kls_JS
accuracy:  0.65306122449
loss:  0.612918235363
match_prob_sf_kls
accuracy:  0.631238634067
loss:  0.709943609271
match_prob_sf_kls_JS
accuracy:  0.632451000202
loss:  0.638863099676
match_prob_adj_kls
accuracy:  0.679531218428
loss:  0.620431497622
match_prob_adj_kls_JS
accuracy:  0.67973327945
loss:  0.608389757952
lm columns:  ['elo_diff']
accuracy:  0.690644574662
loss:  0.578209904735
lm columns:  ['sf_elo_diff']
accuracy:  0.683774499899
loss:  0.584881254079
lm columns:  ['elo_diff_538']
accuracy:  0.692463123863
loss:  0.577909913273
lm columns:  ['sf_elo_diff_538']
accuracy:  0.690846635684
loss:  0.582051444119
lm columns:  ['elo_diff', 'sf_elo_diff']
accuracy:  0.696908466357
loss:  0.574712255606
lm columns:  ['elo_diff_538', 'sf_elo_diff_538']
accuracy:  0.693069306931
loss:  0.573451613361


In [8]:
# cols = [['elo_diff'],['sf_elo_diff'],['elo_diff_538'],['sf_elo_diff_538'],
#         ['elo_diff','sf_elo_diff'],['elo_diff_538','sf_elo_diff_538'],\
#         ['elo_diff','sf_elo_diff','match_z_kls'],\
#         ['elo_diff_538','sf_elo_diff_538','match_z_kls']]
# probs = [u'match_prob_kls',u'match_prob_kls_JS', u'match_prob_sf_kls', u'match_prob_sf_kls_JS',
#         'match_prob_adj_kls','match_prob_adj_kls_JS']
# probs = [u'match_prob_kls',u'match_prob_kls_JS', u'match_prob_sf_kls', u'match_prob_sf_kls_JS',
#         'match_prob_adj_kls','match_prob_adj_kls_JS']
# n_splits = 5
# validate_results(sub_df,probs=probs,lm_columns=cols,n_splits=n_splits)

In [12]:
#df.to_csv('../my_data/elo_atp_matches_21st_century_9_12.csv')

In [21]:
# try out different weightings of normalized surface serve stats...
# probs = []
# for p in [.1,.2,.3,.4,.5,.6,.7,.8,.9]:
#     wgt_s0 = (1-p)*df['p0_s_pct_JS'] + p*df['p0_sf_s_pct_JS']
#     wgt_s1 = (1-p)*df['p1_s_pct_JS'] + p*df['p1_sf_s_pct_JS']
#     wgt_r0 = (1-p)*df['p0_r_pct_JS'] + p*df['p0_sf_r_pct_JS']
#     wgt_r1 = (1-p)*df['p1_r_pct_JS'] + p*df['p1_sf_r_pct_JS']
#     df['p0_s_wgt'+str(p)] = df['tny_stats']+(wgt_s0-df['avg_52_s']) - (wgt_r1-df['avg_52_r'])
#     df['p1_s_wgt'+str(p)] = df['tny_stats']+(wgt_s1-df['avg_52_s']) - (wgt_r0-df['avg_52_r'])
    
#     df['match_prob_wgt_'+str(p)] = [matchProb(row['p0_s_wgt'+str(p)],1-row['p1_s_wgt'+str(p)]) \
#                                     for i,row in df.iterrows()]
#     probs.append('match_prob_wgt_'+str(p))

In [161]:
# one-time fix to transfer over the elo_diff's
# import pandas as pd
# import numpy as np

# df_og = pd.read_csv('../my_data/elo_pbp_with_surface_8_22.csv')
# del df_og['Unnamed: 0']
# df = pd.read_csv('../my_data/feature_df_pbp3_8_23_alphas.csv')
# del df['Unnamed: 0']

# elo_dict = dict(zip(range(len(df_og)),df_og['elo_diff']))
# s_elo_dict = dict(zip(range(len(df_og)),df_og['s_elo_diff']))

# df['elo_diff'] = [elo_dict[m_id] for m_id in df['match_id']]
# df['s_elo_diff'] = [s_elo_dict[m_id] for m_id in df['match_id']]
# df.to_csv('../my_data/feature_df_pbp3_8_23_alphas.csv')

In [ ]:
# from sklearn import linear_model
# from sklearn.model_selection import KFold
# from sklearn.metrics import log_loss,accuracy_score

# n_splits = 5
# kfold = KFold(n_splits=n_splits,shuffle=True)
# scores = np.zeros([3,2,n_splits]);i=0
# cols = ['elo_diff','s_elo_diff']
# for train_ind,test_ind in kfold.split(df):
#     lm = linear_model.LogisticRegression(fit_intercept = True)
#     train_df,test_df = df.loc[train_ind],df.loc[test_ind]
#     lm.fit(train_df[cols].values.reshape([len(train_df),len(cols)]),train_df['winner'])
#     y_preds = lm.predict(test_df[cols].values.reshape([len(test_df),len(cols)]))
#     y_preds2 = test_df['match_prob_kls']>.5
#     y_preds3 = test_df['match_prob_kls_JS']>.5
#     y_probs = lm.predict_proba(test_df[cols].values.reshape([len(test_df),len(cols)]))
#     scores[0][0][i]=accuracy_score(test_df['winner'],y_preds)
#     scores[0][1][i]=log_loss(test_df['winner'],y_probs,labels=[0,1])
#     scores[1][0][i]=accuracy_score(test_df['winner'],y_preds2)
#     scores[1][1][i]=log_loss(test_df['winner'],test_df['match_prob_kls'],labels=[0,1])
#     scores[2][0][i]=accuracy_score(test_df['winner'],y_preds3)
#     scores[2][1][i]=log_loss(test_df['winner'],test_df['match_prob_kls_JS'],labels=[0,1])

    
#     i+=1
# print '% s_elo used in lm fit: ',lm.coef_[0][1]/(lm.coef_[0][0]+lm.coef_[0][1])
# print 'accuracy: ', np.mean(scores[0][0])
# print 'loss: ', np.mean(scores[0][1])

# print 'kls probabilities'
# print 'accuracy: ', np.mean(scores[1][0])
# print 'loss: ', np.mean(scores[1][1])

# print 'kls JS probabilities'
# print 'accuracy: ', np.mean(scores[2][0])
# print 'loss: ', np.mean(scores[2][1])

In [ ]:
# from sklearn import linear_model
# from sklearn.model_selection import KFold
# from sklearn.metrics import log_loss,accuracy_score

# # cols is a list of all column sets to test; compare with kls pre-match forecasts
# def validate_results(df,columns,n_splits=5):
#     kfold = KFold(n_splits=n_splits,shuffle=True)
#     scores = np.zeros([len(columns)+2,2,n_splits]);i=0
#     for train_ind,test_ind in kfold.split(df):
#         lm = linear_model.LogisticRegression(fit_intercept = True)
#         train_df,test_df = df.loc[train_ind],df.loc[test_ind]
        
#         for k,cols in enumerate(columns):
#             lm.fit(train_df[cols].values.reshape([len(train_df),len(cols)]),train_df['winner'])
#             y_preds = lm.predict(test_df[cols].values.reshape([len(test_df),len(cols)]))
#             y_probs = lm.predict_proba(test_df[cols].values.reshape([len(test_df),len(cols)]))
#             scores[k][0][i]=accuracy_score(test_df['winner'],y_preds)
#             scores[k][1][i]=log_loss(test_df['winner'],y_probs,labels=[0,1])
        
#         y_preds2 = test_df['match_prob_kls']>.5
#         y_preds3 = test_df['match_prob_kls_JS']>.5
#         scores[len(columns)][0][i]=accuracy_score(test_df['winner'],y_preds2)
#         scores[len(columns)][1][i]=log_loss(test_df['winner'],test_df['match_prob_kls'],labels=[0,1])
#         scores[len(columns)+1][0][i]=accuracy_score(test_df['winner'],y_preds3)
#         scores[len(columns)+1][1][i]=log_loss(test_df['winner'],test_df['match_prob_kls_JS'],labels=[0,1])
#         i+=1
    
#     for i,cols in enumerate(columns):
#         print 'columns: ',cols
#         #print '% s_elo used in lm fit: ',lm.coef_[0][1]/(lm.coef_[0][0]+lm.coef_[0][1])
#         print 'accuracy: ', np.mean(scores[i][0])
#         print 'loss: ', np.mean(scores[i][1])
    
#     print 'kls probabilities'
#     print 'accuracy: ', np.mean(scores[len(columns)][0])
#     print 'loss: ', np.mean(scores[len(columns)][1])

#     print 'kls JS probabilities'
#     print 'accuracy: ', np.mean(scores[len(columns)+1][0])
#     print 'loss: ', np.mean(scores[len(columns)+1][1])

In [44]:
# from sklearn import linear_model
# from sklearn.model_selection import KFold
# from sklearn.metrics import log_loss,accuracy_score
# # cols is a list of column sets for logistic regression; 
# # probs are model-specific probabilities
# def validate_results(df,probs,lm_columns,n_splits=5):
#     kfold = KFold(n_splits=n_splits,shuffle=True)
#     scores = np.zeros([len(lm_columns)+len(probs),2,n_splits]);i=0
#     for train_ind,test_ind in kfold.split(df):
#         lm = linear_model.LogisticRegression(fit_intercept = True)
#         train_df,test_df = df.loc[train_ind],df.loc[test_ind]
        
#         for j,prob_col in enumerate(probs):
#             y_preds = test_df[prob_col]>.5
#             scores[j][0][i]=accuracy_score(test_df['winner'],y_preds)
#             scores[j][1][i]=log_loss(test_df['winner'],test_df[prob_col],labels=[0,1])
        
#         for k,cols in enumerate(lm_columns):
#             lm.fit(train_df[cols].values.reshape([len(train_df),len(cols)]),train_df['winner'])
#             y_preds = lm.predict(test_df[cols].values.reshape([len(test_df),len(cols)]))
#             y_probs = lm.predict_proba(test_df[cols].values.reshape([len(test_df),len(cols)]))
#             scores[len(probs)+k][0][i]=accuracy_score(test_df['winner'],y_preds)
#             scores[len(probs)+k][1][i]=log_loss(test_df['winner'],y_probs,labels=[0,1])
#         i+=1

#     for j,prob_col in enumerate(probs):
#         print prob_col
#         print 'accuracy: ', np.mean(scores[j][0])
#         print 'loss: ', np.mean(scores[j][1])
    
#     for i,cols in enumerate(lm_columns):
#         print 'lm columns: ',cols
#         print 'accuracy: ', np.mean(scores[len(probs)+i][0])
#         print 'loss: ', np.mean(scores[len(probs)+i][1])

In [ ]:
# current = (0,0)
# for weight in np.arange(11)*.1:
#     accuracy = sum(((1-weight)*df['elo_diff_538']+weight*df['sf_elo_diff_538']>0) == df['winner'])/float(len(df))
#     if accuracy > current[1]:
#         current=weight,accuracy
# print 'best s_elo weight: ',current[0],'accuracy =',current[1]

# sub_df = df[(df['match_prob_adj_kls_JS']>.1) & (df['match_prob_adj_kls_JS']<.9)]
# sub_df = sub_df.reset_index(drop=True)
# ((df['elo_prob_538']>df['elo_prob_538']) & (df['winner']==1))

# df['best_prob'] = np.zeros(len(df))
# elo_ind = ((df['elo_prob_538']>df['sf_elo_prob_538']) & (df['winner']==1) | 
#             (df['elo_prob_538']<df['sf_elo_prob_538']) & (df['winner']==0))
# df['best_prob'].loc[elo_ind] = df['elo_prob_538'].loc[elo_ind]
# df['best_prob'].loc[~elo_ind] = df['sf_elo_prob_538'].loc[~elo_ind]

# log_loss(df['winner'],df['best_prob'])

In [ ]:
# from scipy.stats import norm
# for col in ['match_prob_kls','match_prob_kls_JS','match_prob_adj_kls_JS']:
#     df[col.replace('prob','z')] = norm.ppf(df[col])
# print df.shape

In [39]:
import pandas as pd
import numpy as np

atp_year_list = []
for i in xrange(2010,2018):
    atp_year_list.append(pd.read_csv("../tennis_data/"+TOUR+"/"+TOUR+"_matches_{0}.csv".format(i)))
df = pd.concat(atp_year_list, ignore_index = True)

sub = atp_all_matches
higher_ind = sub['winner_rank']<sub['loser_rank']
lower_ind = sub['winner_rank']>sub['loser_rank']

len(sub[higher_ind])/float(len(sub))

0.6646747648902821